In [1]:
from unityagents import UnityEnvironment
import numpy as np
import gym
import random
import torch
import numpy as np
import time
from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent

In [2]:
## Load Envtt with Visualization
env = UnityEnvironment(file_name='/home/steffen/workspace/deep_reinforcement_learning/projects/p3_collab_compet/Tennis_Linux/Tennis.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
agent_0 = Agent(state_size, action_size, random_seed=0,id=0)
agent_1 = Agent(state_size, action_size, random_seed=0, id=1)
agent_0.actor_local.load_state_dict(torch.load('multi_final_weight_actor1.pth'))
agent_0.critic_local.load_state_dict(torch.load('multi_final_weight_critic1.pth'))
agent_1.actor_local.load_state_dict(torch.load('multi_final_weight_actor2.pth'))
agent_1.critic_local.load_state_dict(torch.load('multi_final_weight_critic2.pth'))

<All keys matched successfully>

In [6]:
scores = []
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores_episode = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        action_0 = agent_0.act(states[0], noise=0)    
        action_1 = agent_1.act(states[1], noise=0)
        actions = np.concatenate((action_0, action_1), axis=0).flatten()
        env_info = env.step(actions)[brain_name]           
        next_states = env_info.vector_observations         
        rewards = env_info.rewards                         
        dones = env_info.local_done                        
        scores_episode += env_info.rewards                        
        states = next_states                              
        if np.any(dones):                                 
            break
    scores.append(np.max(scores_episode))
print("Scores are: ", scores)

Scores are:  [1.9000000283122063, 2.600000038743019, 2.7000000402331352, 0.9000000134110451, 0.20000000298023224]


In [7]:
env.close()